## Misc imports

In [1]:
import os
import cv2
import random
from tqdm import tqdm

## import libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_curve, precision_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers

## Data Loading And Preprocessing

In [3]:
img_size = 200

datas = os.listdir(os.path.join("Data", "train"))
size_of_data = len(os.listdir(os.path.join("Data", "train")))
images_train = []
images_test = []
images_val = []

class_num = -1
for data in datas:
    class_num += 1
    for img in tqdm(os.listdir(os.path.join("Data", "train", data)), desc=f"train data {data} --->"):
        img_np = cv2.resize(cv2.imread(os.path.join(os.path.join("Data", "train", data), img), cv2.COLOR_BGR2RGB).astype(np.float32) / 255., (img_size, img_size))
        images_train.append(
            {
                "Image": img_np,
                "Class": class_num,
                class_num: data
            })
    for img in tqdm(os.listdir(os.path.join("Data", "test", data)), desc=f"test data {data} --->"):
        img_np = cv2.resize(cv2.imread(os.path.join(os.path.join("Data", "test", data), img), cv2.COLOR_BGR2RGB).astype(np.float32) / 255., (img_size, img_size))
        images_test.append(
            {
                "Image": img_np,
                "Class": class_num,
                class_num: data
            })
    for img in tqdm(os.listdir(os.path.join("Data", "valid", data)), desc=f"val data {data} --->"):
        img_np = cv2.resize(cv2.imread(os.path.join(os.path.join("Data", "valid", data), img), cv2.COLOR_BGR2RGB).astype(np.float32) / 255., (img_size, img_size))
        images_val.append(
            {
                "Image": img_np,
                "Class": class_num,
                class_num: data
            })
    if class_num == 5:
        class_num += 1
        break

train data ABBOTTS BABBLER --->:   0%|          | 0/163 [00:00<?, ?it/s]

val data AFRICAN FIREFINCH --->: 100%|██████████| 5/5 [00:00<00:00, 84.75it/s]


## Split And Encoding


In [4]:
X_train = []
y_train = []
for data in images_train:
    X_train.append(data["Image"])
    y_train.append(data["Class"])
X_test = []
y_test = []
for data in images_test:
    X_test.append(data["Image"])
    y_test.append(data["Class"])
X_val = []
y_val = []
for data in images_val:
    X_val.append(data["Image"])
    y_val.append(data["Class"])


X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)
X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)
X_val = np.array(X_val).reshape(-1, img_size, img_size, 3)
y_val = np.array(y_val)

In [5]:
y_train_encoded = to_categorical(y_train, num_classes=class_num)
y_test_encoded = to_categorical(y_test, num_classes=class_num)
y_val_encoded = to_categorical(y_test, num_classes=class_num)

In [6]:
print(X_train.shape)
print(y_train.shape)
print(y_train_encoded.shape)
print(X_test.shape)
print(y_test.shape)
print(y_test_encoded.shape)
print(X_val.shape)
print(y_val.shape)
print(y_val_encoded.shape)

(948, 200, 200, 3)
(948,)
(948, 6)
(30, 200, 200, 3)
(30,)
(30, 6)
(30, 200, 200, 3)
(30,)
(30, 6)


In [7]:
def build_efficientnet_model(img_size, class_num):
    # Define the input shape
    img_shape = (img_size, img_size, 3)
    
    # Load pre-trained EfficientNet model
    efficient_net = tf.keras.applications.EfficientNetB0(include_top=False,
                                            weights='imagenet',
                                            input_shape=img_shape)

    # Freeze layers in the base model
    for layer in efficient_net.layers:
        layer.trainable = False

    # Create a sequential model
    model = Sequential()

    # Add the base model
    model.add(efficient_net)

    # Add a global average pooling layer
    model.add(layers.GlobalAveragePooling2D())

    # Add a flatten layer
    model.add(layers.Flatten())

    # Add final classification layers
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(class_num, activation='softmax'))
    
    model.build((None, img_size, img_size, 3))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'precision', 'recall'])

    # Print model summary
    model.summary()

    # visualization_path = 'efficientnet_model_visualization.png'
    # tf.keras.utils.plot_model(model, to_file=visualization_path, show_shapes=True, show_layer_names=True)

    # print(f"Model visualization saved to {visualization_path}")

    return model

In [8]:
single_model = build_efficientnet_model(img_size, class_num)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,379,049 (16.70 MB)

 Trainable params: 329,478 (1.26 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [11]:
model_history = single_model.fit(X_train, y_train_encoded, epochs=2, validation_data=(X_val, y_val_encoded))

Epoch 1/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 24s 794ms/step - accuracy: 0.2087 - loss: 1.7884 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.1667 - val_loss: 1.7955 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 23s 754ms/step - accuracy: 0.1754 - loss: 1.7912 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.1667 - val_loss: 1.7974 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00


In [12]:
prediction = single_model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [13]:
pred = np.argmax(prediction, axis=1)

In [14]:
accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)

Accuracy: 0.16666666666666666
